# Raw Data to Dataset Conversion

In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
import os

def get_folder_names(directory):
    folder_names = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
    return folder_names

# Specify the directory
directory_path = '/kaggle/input/20ng-bydate/20news-bydate-train'
folders = get_folder_names(directory_path)
print(folders)

['soc.religion.christian', 'talk.politics.misc', 'rec.motorcycles', 'talk.politics.mideast', 'sci.electronics', 'sci.space', 'rec.sport.baseball', 'misc.forsale', 'talk.politics.guns', 'sci.med', 'comp.sys.mac.hardware', 'rec.sport.hockey', 'comp.sys.ibm.pc.hardware', 'talk.religion.misc', 'sci.crypt', 'comp.windows.x', 'comp.os.ms-windows.misc', 'rec.autos', 'comp.graphics', 'alt.atheism']


In [3]:
import pandas as pd


df = pd.DataFrame(columns=['text','label'])


In [4]:
df

,text,label


In [5]:
import os

def read_text_files(directory):
    text_data = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='latin-1') as file:
            text_data.append(file.read())
    return text_data

    
text_list=[]
label_list=[]
for folder in folders :
    path='/kaggle/input/20ng-bydate/20news-bydate-train/'+folder
    text_files_content = read_text_files(path)
    text_list.extend(text_files_content)
    label_array=[folder.split('.') for i in range(len(text_files_content))]
    label_list.extend(label_array)

for folder in folders :
    path='/kaggle/input/20ng-bydate/20news-bydate-test/'+folder
    text_files_content = read_text_files(path)
    text_list.extend(text_files_content)
    label_array=[folder.split('.') for i in range(len(text_files_content))]
    label_list.extend(label_array)


In [6]:
len(text_list)

18846

In [7]:
len(label_list)

18846

In [8]:
df['text']=text_list
df['label']=label_list

In [9]:
df

,text,label
0,From: kene@acs.bu.edu (Kenneth Engel)\nSubject...,"[soc, religion, christian]"
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,"[soc, religion, christian]"
2,From: conditt@tsd.arlut.utexas.edu (Paul Condi...,"[soc, religion, christian]"
3,From: djohnson@cs.ucsd.edu (Darin Johnson)\nSu...,"[soc, religion, christian]"
4,From: hedrick@cs.rutgers.edu\nSubject: Re: Doi...,"[soc, religion, christian]"
...,...,...
18841,From: ednclark@kraken.itc.gu.edu.au (Jeffrey C...,"[alt, atheism]"
18842,From: healta@saturn.wwc.edu (Tammy R Healy)\nS...,"[alt, atheism]"
18843,From: kempmp@phoenix.oulu.fi (Petri Pihko)\nSu...,"[alt, atheism]"
18844,From: sandvik@newton.apple.com (Kent Sandvik)\...,"[alt, atheism]"


In [10]:
shuffled_df = df.sample(frac=1).reset_index(drop=True)

In [11]:
shuffled_df

,text,label
0,From: gary@ke4zv.uucp (Gary Coffman)\nSubject:...,"[sci, space]"
1,From: triumvir@cco.caltech.edu (Martin W. Lin)...,"[comp, sys, mac, hardware]"
2,From: tdawson@engin.umich.edu (Chris Herringsh...,"[comp, graphics]"
3,From: bskendig@netcom.com (Brian Kendig)\nSubj...,"[talk, religion, misc]"
4,From: rmohns@vax.clarku.edu\nSubject: RE: Need...,"[comp, os, ms-windows, misc]"
...,...,...
18841,From: mlf@unl.edu (mary flaglelee)\nSubject: W...,"[misc, forsale]"
18842,From: 02106@ravel.udel.edu (Samuel Ross)\nSubj...,"[misc, forsale]"
18843,From: brian@meaddata.com (Brian Curran)\nSubje...,"[rec, sport, baseball]"
18844,From: dmeyers@mal-s2.gatech.edu (Dave Meyers)\...,"[talk, politics, guns]"


In [ ]:
# encode categories into labele columns
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(shuffled_df.label)
labels

In [ ]:
df = pd.concat([shuffled_df[['text']], pd.DataFrame(labels)], axis=1)
df.columns = ['text'] + list(mlb.classes_)
df.head(4)

In [ ]:
# For our task we remove the categories with small number of articles
columns = df.columns
# Find the frequencies of the articles in every category
categor_freq = df[columns[2:]].sum()/df.shape[0]
categor_freq

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (15,3)
plt.bar(categor_freq.index, categor_freq.values)
_ = plt.xticks(rotation = 45)

In [ ]:
# Drop the categories with low frequencies
signif_cols = categor_freq.index[categor_freq.values>0.05]
df0 = df[['text']+list(signif_cols)]
# Remove articles with zeros in all the categories
df0 = df0.loc[df0[signif_cols].sum(axis=1)>0]
df0.head(4)

In [ ]:
df0

In [ ]:
# Save the preprocessed data
df0.to_csv("/kaggle/working/20ng_bydate.tsv", sep="\t", header=True, index=False)